In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.optimize import minimize
import json
import math
import itertools 
import copy

from utils import generate_structures

### Loading Data 

In [2]:
# LOADING RESPONSE MATRICES WITH N ROOMS * N AGENTS * N RESPONSES EACH. Note: Rooms = 21 for all but independent_unknown condition (here 22 rooms)
# independent_unknown = np.array(np.loadtxt('processed_data_and_figures/exp_3/independent/independent_planet_judgments.txt').reshape(23,3,10))
independent_unknown = np.array(np.loadtxt('processed_data_and_figures/exp_1_one_player_known_structure/main/cond_2_no_struct_full.txt').reshape(51,3,10))

### POSSIBLE STRUCTURES

In [3]:
# preliminaries 
n_trials = 3
n_agents = 3
filtered_adjacency_matrices = generate_structures(n_agents) 

print(filtered_adjacency_matrices)
# todo: add structure_learner.py class to allow for flexible learning with larger graphs 

[array([[0, 0, 0],
       [1, 0, 0],
       [1, 0, 0]]), array([[0, 0, 0],
       [1, 0, 0],
       [1, 1, 0]]), array([[0, 0, 0],
       [1, 0, 1],
       [1, 0, 0]]), array([[0, 0, 0],
       [1, 0, 1],
       [1, 1, 0]]), array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]]), array([[0, 0, 1],
       [1, 0, 0],
       [1, 1, 0]]), array([[0, 0, 1],
       [1, 0, 1],
       [1, 0, 0]]), array([[0, 0, 1],
       [1, 0, 1],
       [1, 1, 0]]), array([[0, 1, 0],
       [1, 0, 0],
       [1, 0, 0]]), array([[0, 1, 0],
       [1, 0, 0],
       [1, 1, 0]]), array([[0, 1, 0],
       [1, 0, 1],
       [1, 0, 0]]), array([[0, 1, 0],
       [1, 0, 1],
       [1, 1, 0]]), array([[0, 1, 1],
       [1, 0, 0],
       [1, 0, 0]]), array([[0, 1, 1],
       [1, 0, 0],
       [1, 1, 0]]), array([[0, 1, 1],
       [1, 0, 1],
       [1, 0, 0]]), array([[0, 1, 1],
       [1, 0, 1],
       [1, 1, 0]])]


## Structure Learners
For now, each structure learner will use additive combination functions.

#### Note: 
Responses are transformed to 1-7 scale for structure modelling 

In [4]:
# PRELIMINARIES

# Hyperparams
theta = 0.9 # accuracy of execution (of the deterministic policy implemented by a learner)

# Function returning all edges (from row i to colum j) of input DAG 
def get_edges(adjacency_matrix):
    return [(i,j) for i,l in enumerate(adjacency_matrix) for j,v in enumerate(l) if v]

def check_self_change(self_t, self_prev_t, theta, l, s):
    ''' self transition'''
    if self_t == self_prev_t:
        l *= theta + (1-theta)/s
    else:
        l *= (1-theta)/s
    return l

#### 1. Categorical change
If X and Y change belief, Z will change (from state $s$ to another state $s' \neq s$) with probability theta

In [5]:
def categorical_change_likelihood(agents, xx, yy, zz, theta, adjacency_matrix, s=7):
    
    assert len(xx) == len(yy) == len(zz)
    
    l = 1
    
    for t in range(len(xx)):  # looping over all trials 
        if t == 0:
            l *= (1/s)**3     # initial random prediction for all three agents thus (1/s)**3
            
        elif t == 1:          # self transition at t=1 as no change recorded yet in others
            
            l *= check_self_change(xx[t], xx[t-1], theta, l, s)
            l *= check_self_change(yy[t], yy[t-1], theta, l, s)
            l *= check_self_change(zz[t], zz[t-1], theta, l, s)
            
        elif t > 1:  # now consider change in others from prev2 and prev time step to predict current time step
            
            edges = get_edges(adjacency_matrix) 

            [edgeXY, edgeXZ, edgeYX, edgeYZ, edgeZX, edgeZY] = list(np.repeat(False, 6))  # possible edges initially all false

             # determine which edges are True based on edges in input DAG 
            if (0, 1) in edges:
                edgeXY = True 
            if (0, 2) in edges:
                edgeXZ = True 
            if (1, 0) in edges:
                edgeYX = True 
            if (1, 2) in edges:
                edgeYZ = True 
            if (2, 0) in edges:
                edgeZX = True  
            if (2, 1) in edges:
                edgeZY = True  
            
            # for now store in dict 
            edges = {'edgeXY': edgeXY, 
                     'edgeXZ': edgeXZ, 
                     'edgeYX': edgeYX, 
                     'edgeYZ': edgeYZ, 
                     'edgeZX': edgeZX, 
                     'edgeZY': edgeZY}

            for agent in agents:  # now looping over all agents and multiplying likelihood accordingly 
            
                l *= categorical_change_step(agent, 
                                             xx[t-2], 
                                             xx[t-1], 
                                             xx[t], 
                                             yy[t-2], 
                                             yy[t-1], 
                                             yy[t],
                                             zz[t-2], 
                                             zz[t-1],
                                             zz[t],
                                             edges, 
                                             theta, 
                                             s)

    return l
            
     
# at each step compute probabilities  
def categorical_change_step(agent,
                            prevX2, 
                            prevX,
                            currX,
                            prevY2, 
                            prevY,
                            currY,
                            prevZ2, 
                            prevZ, 
                            currZ,
                            edges,
                            theta, 
                            s=7):
   
    l = 1
    
    # check direction of change
    changeX = prevX - prevX2
    changeY = prevY - prevY2
    changeZ = prevZ - prevZ2
   
    # Agent X
    if agent == 'X':
        # with both edges: 
        if edges['edgeYX'] and edges['edgeZX']:
            # change 
            if changeY + changeZ != 0:     
                if prevX != currX:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s
            # else additive combination function reveals no change:
            elif changeY + changeZ == 0:
                if prevX == currX: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # one edge Y->X
        elif edges['edgeYX']:
            if changeY != 0:
                # change 
                if prevX != currX:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s          
            elif changeY == 0:
                # no change 
                if prevX == currX: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
            
        # one edge Z->X
        elif edges['edgeZX']:
            if changeZ != 0:
                # change 
                if prevX != currX:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s       
            elif changeZ == 0:
                # no change 
                if prevX == currX: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # no edges
        elif edges['edgeYX'] == False and edges['edgeZX'] == False:
            if prevX == currX: 
                l *= theta + (1-theta)/s 
            else:
                l *= (1-theta)/s 
            return l
            
    # Agent Y
    if agent == 'Y':
        # with both edges: 
        if edges['edgeXY'] and edges['edgeZY']:
            # change 
            if changeX + changeZ != 0:     
                if prevY != currY:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s
            # else additive combination function reveals no change:
            elif changeX + changeZ == 0:
                if prevY == currY: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # one edge X->Y
        elif edges['edgeXY']:
            if changeX != 0:
                # change 
                if prevY != currY:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s           
            elif changeX == 0:
                # no change 
                if prevY == currY: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
            
        # one edge Z->Y
        elif edges['edgeZY']:
            if changeZ != 0:
                # change 
                if prevY != currY:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s         
            elif changeZ == 0:
                # no change 
                if prevY == currY: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # no edges
        elif edges['edgeXY'] == False and edges['edgeZY'] == False:
            if prevY == currY: 
                l *= theta + (1-theta)/s 
            else:
                l *= (1-theta)/s 
            return l
    
    # Agent Z
    if agent == 'Z':
        # with both edges: 
        if edges['edgeYZ'] and edges['edgeXZ']:
            # change 
            if changeX + changeY != 0:     
                if prevZ != currZ:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s
            # else additive combination function reveals no change:
            elif changeX + changeY == 0:
                if prevZ == currZ: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # one edge X->Z
        elif edges['edgeXZ']:
            if changeX != 0:
                # change 
                if prevZ != currZ:
                    l *= theta/(s-prevZ + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s         
            elif changeX == 0:
                # no change 
                if prevZ == currZ: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
            
        # one edge Y->Z
        elif edges['edgeYZ']:
            if changeY != 0:
                # change 
                if prevZ != currZ:
                    l *= theta/(s-1) + (1-theta)/s
                else:
                    l *= (1-theta)/s        
            elif changeY == 0:
                # no change 
                if prevZ == currZ: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # no edges
        elif edges['edgeXZ'] == False and edges['edgeYZ'] == False:
            if prevZ == currZ: 
                l *= theta + (1-theta)/s 
            else:
                l *= (1-theta)/s 
            return l

### Ordinal change
If X and Y changed, Z will change in the same direction with probability theta

In [6]:
def ordinal_change_likelihood(agents, xx, yy, zz, theta, adjacency_matrix, s=7):
    
    assert len(xx) == len(yy) == len(zz)
    
    l = 1
    
    for t in range(len(xx)):  # looping over all trials 
        
        if t == 0:
            l *= (1/s)**3     # initial random prediction for all three agents thus (1/s)**3

        elif t == 1:          # self transition at t=1 as no change recorded yet in others

            l *= check_self_change(xx[t], xx[t-1], theta, l, s)
            l *= check_self_change(yy[t], yy[t-1], theta, l, s)
            l *= check_self_change(zz[t], zz[t-1], theta, l, s)

        elif t > 1:  # now consider change in others from prev2 and prev time step to predict current time step
            edges = get_edges(adjacency_matrix) 

            [edgeXY, edgeXZ, edgeYX, edgeYZ, edgeZX, edgeZY] = list(np.repeat(False, 6))  # possible edges initially all false

            # determine which edges are True based on edges in input DAG 
            if (0, 1) in edges:
                edgeXY = True 
            if (0, 2) in edges:
                edgeXZ = True 
            if (1, 0) in edges:
                edgeYX = True 
            if (1, 2) in edges:
                edgeYZ = True 
            if (2, 0) in edges:
                edgeZX = True  
            if (2, 1) in edges:
                edgeZY = True  

            # for now store in dict 
            edges = {'edgeXY': edgeXY, 
                     'edgeXZ': edgeXZ, 
                     'edgeYX': edgeYX, 
                     'edgeYZ': edgeYZ, 
                     'edgeZX': edgeZX, 
                     'edgeZY': edgeZY}

            for agent in agents:  # now looping over all agents and multiplying likelihood accordingly 

                l *= ordinal_change_step(agent, 
                                             xx[t-2], 
                                             xx[t-1], 
                                             xx[t], 
                                             yy[t-2], 
                                             yy[t-1], 
                                             yy[t],
                                             zz[t-2], 
                                             zz[t-1],
                                             zz[t],
                                             edges, 
                                             theta, 
                                             s)

    return l
            
     
            
# at each step compute probabilities  
def ordinal_change_step(agent,
                        prevX2, 
                        prevX,
                        currX,
                        prevY2, 
                        prevY,
                        currY,
                        prevZ2, 
                        prevZ, 
                        currZ,
                        edges,
                        theta, 
                        s=7):
   
    l = 1
    
    # check direction of change
    changeX = prevX - prevX2
    changeY = prevY - prevY2
    changeZ = prevZ - prevZ2
   
    # Agent X
    if agent == 'X':
        # with both edges: 
        if edges['edgeYX'] and edges['edgeZX']:
            # positive change 
            if changeY + changeZ > 0:     
                if prevX <= currX:
                    l *= theta/(s-prevX + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s
            # negative change 
            elif changeY + changeZ < 0: 
                if prevX >= currX:
                    l *= theta/prevX + (1-theta)/s 
                else:
                    l *= (1-theta)/s
            # else additive combination function reveals no change:
            elif changeY + changeZ == 0:
                if prevX == currX: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # one edge Y->X
        elif edges['edgeYX']:
            if changeY > 0:
                # positive change 
                if prevX <= currX:
                    l *= theta/(s-prevX + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s       
            elif changeY < 0:
                # negative change 
                if prevX >= currX:
                    l *= theta/prevX + (1-theta)/s 
                else:
                    l *= (1-theta)/s    
            elif changeY == 0:
                # no change 
                if prevX == currX: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
            
        # one edge Z->X
        elif edges['edgeZX']:
            if changeZ > 0:
                # positive change 
                if prevX <= currX:
                    l *= theta/(s-prevX + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s       
            elif changeZ < 0:
                # negative change 
                if prevX >= currX:
                    l *= theta/prevX + (1-theta)/s 
                else:
                    l *= (1-theta)/s    
            elif changeZ == 0:
                # no change 
                if prevX == currX: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # no edges
        elif edges['edgeYX'] == False and edges['edgeZX'] == False:
            if prevX == currX: 
                l *= theta + (1-theta)/s 
            else:
                l *= (1-theta)/s 
            return l
            
    # Agent Y
    if agent == 'Y':
        # with both edges: 
        if edges['edgeXY'] and edges['edgeZY']:
            # positive change 
            if changeX + changeZ > 0:     
                if prevY <= currY:
                    l *= theta/(s-prevY + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s
            # negative change 
            elif changeX + changeZ < 0: 
                if prevY >= currY:
                    l *= theta/prevY + (1-theta)/s 
                else:
                    l *= (1-theta)/s
            # else additive combination function reveals no change:
            elif changeX + changeZ == 0:
                if prevY == currY: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # one edge X->Y
        elif edges['edgeXY']:
            if changeX > 0:
                # positive change 
                if prevY <= currY:
                    l *= theta/(s-prevY + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s       
            elif changeX < 0:
                # negative change 
                if prevY >= currY:
                    l *= theta/prevY + (1-theta)/s 
                else:
                    l *= (1-theta)/s    
            elif changeX == 0:
                # no change 
                if prevY == currY: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
            
        # one edge Z->Y
        elif edges['edgeZY']:
            if changeZ > 0:
                # positive change 
                if prevY <= currY:
                    l *= theta/(s-prevY + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s       
            elif changeZ < 0:
                # negative change 
                if prevY >= currY:
                    l *= theta/prevY + (1-theta)/s 
                else:
                    l *= (1-theta)/s    
            elif changeZ == 0:
                # no change 
                if prevY == currY: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # no edges
        elif edges['edgeXY'] == False and edges['edgeZY'] == False:
            if prevY == currY: 
                l *= theta + (1-theta)/s 
            else:
                l *= (1-theta)/s 
            return l
    
    # Agent Z
    if agent == 'Z':
        # with both edges: 
        if edges['edgeYZ'] and edges['edgeXZ']:
            # positive change 
            if changeX + changeY > 0:     
                if prevZ <= currZ:
                    l *= theta/(s-prevZ + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s
            # negative change 
            elif changeX + changeY < 0: 
                if prevZ >= currZ:
                    l *= theta/prevZ + (1-theta)/s 
                else:
                    l *= (1-theta)/s
            # else additive combination function reveals no change:
            elif changeX + changeY == 0:
                if prevZ == currZ: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # one edge X->Z
        elif edges['edgeXZ']:
            if changeX > 0:
                # positive change 
                if prevZ <= currZ:
                    l *= theta/(s-prevZ + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s       
            elif changeX < 0:
                # negative change 
                if prevZ >= currZ:
                    l *= theta/prevZ + (1-theta)/s 
                else:
                    l *= (1-theta)/s    
            elif changeX == 0:
                # no change 
                if prevZ == currZ: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
            
        # one edge Y->Z
        elif edges['edgeYZ']:
            if changeY > 0:
                # positive change 
                if prevZ <= currZ:
                    l *= theta/(s-prevZ + 1) + (1-theta)/s 
                else:
                    l *= (1-theta)/s       
            elif changeY < 0:
                # negative change 
                if prevZ >= currZ:
                    l *= theta/prevZ + (1-theta)/s 
                else:
                    l *= (1-theta)/s    
            elif changeY == 0:
                # no change 
                if prevZ == currZ: 
                    l *= theta + (1-theta)/s 
                else:
                    l *= (1-theta)/s        
            # return likelihood
            return l
        
        # no edges
        elif edges['edgeXZ'] == False and edges['edgeYZ'] == False:
            if prevZ == currZ: 
                l *= theta + (1-theta)/s 
            else:
                l *= (1-theta)/s 
            return l

### Computing Posteriors over structures based on participant planet judgments

In [7]:
# POSTERIOR STRUCTURE LL FOR ONE TRIAL (ie one triad)
def structure_posterior(agents, xx, yy, zz, structure_learner, theta, adjacency_matrices): 
    # transform to desired scale
    xx = xx + 4
    yy = yy + 4
    zz = zz + 4
    
    unnorm_post = {}
    N = 0
    n = 0
    
    for adjacency_matrix in adjacency_matrices:
        
        likelihood = structure_learner(agents, xx, yy, zz, theta, adjacency_matrix, s=7)
        unnorm_post[n] = likelihood
        N += likelihood
        n += 1
        
    post = {s:np.round(v/N,5) for s,v in unnorm_post.items()}
    post_min = {}
    vals = list(post.values())

    return post, vals

In [8]:
# # GETTING POSTERIORS FOR EACH TRIAD AT t10

# # summary of posteriors f
# pred_posteriors = []

# agents = ['X', 'Y', 'Z']

# # insert condition here 
# for trial in range(independent_unknown.shape[0]-1):
    
#     xx = independent_unknown[trial, 0, :]
#     yy = independent_unknown[trial, 1, :]
#     zz = independent_unknown[trial, 2, :]
    
#     # ordinal for now 
#     post = structure_posterior(agents, xx, yy, zz, ordinal_change_likelihood, .9, filtered_adjacency_matrices)
#     pred_posteriors.append(post[1])
    

# best_posteriors = []
# for trial in pred_posteriors:
#     best_posteriors.append(filtered_adjacency_matrices[trial.index(np.max(trial))])

    
# raw_posteriors = []
# for trial in pred_posteriors:
#     raw_posteriors.append(trial)

# # weighted posteriors
# weighted_posterior = np.zeros((3, 3))
# weighted_posteriors = np.zeros((22, 3, 3)) # change dependent on number of triads
# average_matrices = np.zeros((3, 3))

# room_index = 0

# for room in pred_posteriors:
#     for weight, struct in zip(room, filtered_adjacency_matrices):
#         for column in range(3):
#             weighted_posterior[0, column] += struct[0, column] * weight * 1/len(pred_posteriors)
#             weighted_posterior[1, column] += struct[1, column] * weight * 1/len(pred_posteriors)
#             weighted_posterior[2, column] += struct[2, column] * weight * 1/len(pred_posteriors)
            
#             weighted_posteriors[room_index, 0, column] += struct[0, column] * weight 
#             weighted_posteriors[room_index, 1, column] += struct[1, column] * weight 
#             weighted_posteriors[room_index, 2, column] += struct[2, column] * weight 
            
#             average_matrices[0, column] += struct[0, column] * 1/len(pred_posteriors)/len(filtered_adjacency_matrices)
#             average_matrices[1, column] += struct[1, column] * 1/len(pred_posteriors)/len(filtered_adjacency_matrices)
#             average_matrices[2, column] += struct[2, column] * 1/len(pred_posteriors)/len(filtered_adjacency_matrices)
#             print
            
#     room_index += 1

# best_post_dict = {}
# count = 0
# for best_post in best_posteriors:
#     best_post_dict[str(count)] = None
#     formatted = best_posteriors[count].reshape(9,1)
#     formatted = [i[0] for i in formatted]
#     best_post_dict[str(count)] = formatted
#     count += 1

##### Plotting weighted posterior structures per trial 

In [9]:
# def show_structure(structures):
    
#     ids = []
#     structure_plot = []
    
#     for trial in range(len(structures)):
#         ids.append(trial)
#         structure_plot.append(structures[trial])
    

#     ids.append('Aggregate')
#     average_matrix = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    
#     for matrix in structures:
#         average_matrix = average_matrix + matrix
#     average_matrix = average_matrix / len(structures)
    
#     structure_plot.append(average_matrix)
#     fig, axs = plt.subplots(1, len(ids), figsize=(165,5)) 
   
#     for i in range(len(ids)):   

#         matrix_i = copy.deepcopy(structure_plot[i])
#         matrix_i[matrix_i == 0] = 0.01 # just to make sure all edges are drawn and graphs have same dimensions
        
#         pos = {1: (1, 0.9), 0: (0.9, 1), 2: (1.1, 1)} 

#         # draw DAG graph from adjacency matrix 
#         gr = nx.from_numpy_matrix( matrix_i, create_using=nx.DiGraph)
#         weights = nx.get_edge_attributes(gr, "weight")
       
#         # adding nodes 
#         all_rows = range(0, matrix_i.shape[0])
#         for n in all_rows:
#             gr.add_node(n)
        
#         # getting edges 
#         edges = gr.edges()
          
#         # weight and color of edges 
#         alphas = [weights[edge] * 2 for edge in edges]
#         colors = [tuple(np.repeat(1-weights[edge],3)) + (weights[edge],) for edge in edges]
#         colors[3] = (0,128/255,0,1.0)
#         colors[6] = (0,128/255,0,1.0)
      
#         # draw graph 
#         nx.draw(gr, 
#                 pos, 
#                 ax=axs[i],
#                 edgecolors='black', 
#                 node_color='white', 
#                 arrowsize=20,
#                 arrowstyle='simple',
#                 node_size=2000, 
#                 labels={0: "A", 1: "B", 2: "C"},
#                 font_weight='bold',
#                 linewidths=2,
#                 with_labels=True,
#                 connectionstyle="arc3,rad=0.12",     
#                 edge_color=colors)

#         # set title 
#         if i <= len(ids) - 1:
#             axs[i].set_title(str(i))
#         elif i == len(ids):
#             axs[i].set_title(str(ids[i]), size=10,  y=1)
         
#     plt.tight_layout()
# #     plt.savefig('subject_predictions.pdf') 
#     plt.show()
#     return average_matrix

### Model Comparison 

#### Fit per participant (posterior G probability and then posterior edge prob)

In [10]:
# g_loglikelihoods = []
# g_taus = []

# edge_loglikelihoods = []
# edge_taus = []


# # FIRST POSTERIOR PROBABILITY OF G 
# def fit_structure_posteriors(parameters, pred, select, filtered_adjacency_matrices):
    
#     tau = parameters[0]
#     select_probs = np.zeros(len(filtered_adjacency_matrices)) # hot vector of size 64
#     pred_probs = np.zeros(len(filtered_adjacency_matrices))
#     struct_index = 0
    
#     if np.sum(select) == 0: # this is only because on trial includes an empty structure selection, probably a bug so just resetting to default given prior to selection
#         select = np.array([[0,0,0],[1,0,0],[1,0,0]])
        
#     for struct in filtered_adjacency_matrices:
#         if list([list(i) for i in select]) == list([list(i) for i in struct]):
#             select_probs[struct_index] = 1
            
#         if list([list(i) for i in pred]) == list([list(i) for i in struct]):
#             pred_probs[struct_index] = 1
        
#         struct_index += 1
# #     pred_probs = pred
    
#     softmax_probs = math.e**(np.array(pred_probs)*tau)/np.sum(math.e**(np.array(pred_probs)*tau))
#     fitted_prob = np.sum([a * b for a, b in zip(softmax_probs, select_probs)])


#     return -np.log(fitted_prob) 
# #     return 1



# # FITTING 
# for pred, select in zip(best_posteriors, independent_unknown_structure):
#     tau = 1
#     bounds = [(0.001, 1000.00)]
#     parameter_guesses = [tau]

#     res = minimize(fit_structure_posteriors, parameter_guesses, args=(pred, select, filtered_adjacency_matrices_B_C_only), method='L-BFGS-B', bounds=bounds)
#     fitted_tau = res.x[0]
#     g_taus.append(fitted_tau)

#     g_loglikelihoods.append(-fit_structure_posteriors([fitted_tau], pred, select, filtered_adjacency_matrices))

# print('G PROB')
# print(sum(g_loglikelihoods))




# # SECOND - FIT POSTERIOR EDGE PROBABILITY (PROB THAT EDGE WAS SELECTED )
# def fit_edge_posteriors(parameters, pred, select):
    
#     tau = parameters[0]
#     select_probs = np.zeros(len(pred))
#     struct_index = 0
#     edge_pred = []
#     edge_select = []
    
#     for agent in range(pred.shape[0]):

#         if agent == 0:
#             out_subj1 = select[agent,1]
#             out_subj2 = select[agent,2]
#             edge_select.append(out_subj1)
#             edge_select.append(out_subj2)

#             out_mod1 = pred[agent,1]
#             out_mod2 = pred[agent,2] 
#             edge_pred.append(out_mod1)
#             edge_pred.append(out_mod2)

#         elif agent == 1:
#             out_subj2 = select[agent,2]
#             edge_select.append(out_subj2)
#             out_mod2 = pred[agent,2] 
#             edge_pred.append(out_mod2)

#         elif agent == 2:
#             out_subj1 = select[agent,1]
#             edge_select.append(out_subj1)
#             out_mod1 = pred[agent,1] 
#             edge_pred.append(out_mod1)
    
#     fitted_probs = []
    
#     for a, b in zip(edge_pred, edge_select):
#         softmax_prob = math.e**(a*tau)/(math.e**(a*tau) + math.e**(a*(1-tau)))
#         if b == 0:
#             softmax_prob = 1 - softmax_prob
#         fitted_probs.append(softmax_prob)
        
#     return -np.sum([np.log(p) for p in fitted_probs]) 

     
# # FITTING 
# for pred, select in zip(weighted_posteriors, independent_unknown_structure):
#     tau = 1
#     bounds = [(0.001, 1000.00)]
#     parameter_guesses = [tau]

#     res = minimize(fit_edge_posteriors, parameter_guesses, args=(pred, select), method='L-BFGS-B', bounds=bounds)
#     fitted_tau = res.x[0]
#     edge_taus.append(fitted_tau)
#     edge_loglikelihoods.append(-fit_edge_posteriors([fitted_tau], pred, select))

# print('EDGE PROB')
# print(sum(edge_loglikelihoods))


#### Now compute posterior over structures for each time step given data observed thus far 
(this will be used for the structure-based bayesian learner)

In [40]:
# GETTING POSTERIORS FOR EACH TRIAL 
pred_posteriors = {}

# filtered_adjacency_matrices = [np.array([[0., 0., 0.],
#                                       [1., 0., 0.],
#                                       [1., 0., 0.]]),
#                                np.array([[0., 0., 0.],
#                                       [1., 0., 0.],
#                                       [1., 1., 0.]]),
#                                np.array([[0., 0., 0.],
#                                       [1., 0., 1.],
#                                       [1., 0., 0.]])]

# looping over all trials (triads)
for trial in range(independent_unknown.shape[0]-1):
    pred_posteriors[str(trial)] = {}
    # loopping over time steps starting at t = 2
    for time_step in range(len(independent_unknown[trial, 0, :])):
        # now get data obtained thus far
        xx = independent_unknown[trial, 0, :time_step]
        yy = independent_unknown[trial, 1, :time_step]
        zz = independent_unknown[trial, 2, :time_step]
        post = structure_posterior(agents, xx, yy, zz, ordinal_change_likelihood, .9, filtered_adjacency_matrices)
        pred_posteriors[str(trial)][str(time_step)] = post[1]

# best posterior, and weighted posterior at each time step
best_posteriors = {}
trial_index = 0
for trial in pred_posteriors.values():
    best_posteriors[str(trial_index)] = []
    for time_step in trial.values():
        best_posteriors[str(trial_index)].append(filtered_adjacency_matrices[time_step.index(np.max(time_step))])
    trial_index += 1
best_post_dict = {}
count = 0
for k in best_posteriors.keys():
    best_post_dict[str(count)] = {}
    for time_step in range(10):  
        formatted = best_posteriors[k][time_step].reshape(9,1)
        formatted = [i[0] for i in formatted]
        best_post_dict[str(count)][str(time_step)] = formatted 
    count += 1
     
# with open('inferred_structure/independent_unknown_ordinal_map_per_time_step.txt', 'w') as outfile:
#     json.dump(best_post_dict, outfile)
    
# with open('inferred_structure/exp_1_cond_2_no_struct.txt', 'w') as outfile:
    # json.dump(pred_posteriors, outfile)

In [41]:
print(pred_posteriors)

{'0': {'0': [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625], '1': [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625], '2': [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625], '3': [0.09907, 0.09907, 0.02593, 0.02593, 0.09907, 0.09907, 0.02593, 0.02593, 0.09907, 0.09907, 0.02593, 0.02593, 0.09907, 0.09907, 0.02593, 0.02593], '4': [0.09907, 0.09907, 0.02593, 0.02593, 0.09907, 0.09907, 0.02593, 0.02593, 0.09907, 0.09907, 0.02593, 0.02593, 0.09907, 0.09907, 0.02593, 0.02593], '5': [0.20076, 0.03607, 0.01117, 0.00201, 0.20076, 0.03607, 0.01117, 0.00201, 0.20076, 0.03607, 0.01117, 0.00201, 0.20076, 0.03607, 0.01117, 0.00201], '6': [0.43924, 0.07893, 0.02443, 0.00439, 0.09334, 0.01677, 0.00519, 0.00093, 0.22305, 0.04008, 0.0124, 0.00223, 0.0474, 0.00852, 0.002